In [1]:
import numpy as np
import cv2 
import json
from random import randint, shuffle, uniform, choice
from tqdm import tqdm

In [2]:
DRAWING_HEIGHT, DRAWING_WIDTH = 255, 255
IMG_HEIGHT, IMG_WIDTH = 512, 512
GRID_SIZE = (4, 4)
SCALELIMITS = (0.8, 2.5)
OFFSETLIMIT = (0.3, 0.3)
MARGIN = 10
COLORBOUNDS = (20, 120)
P_SABOTAGE = 0.15
CELL = (IMG_WIDTH // GRID_SIZE[0], IMG_HEIGHT // GRID_SIZE[1])

In [3]:
files = ['circle', 'hexagon', 'octagon', 'square', 'star', 'triangle', 'line']
for i in range(len(files)):
    files[i] = f'./shapes/full_simplified_{files[i]}.ndjson'

In [4]:
drawings = []
for file in tqdm(files):
    with open(file, 'r') as f:
        for line in f:
            drawings.append(json.loads(line))

100%|██████████| 7/7 [00:21<00:00,  3.03s/it]


In [5]:
shuffle(drawings)

In [6]:
def checkBounds(xy):
    return xy[0] >= 0 and xy[0] < IMG_WIDTH and xy[1] >= 0 and xy[1] < IMG_HEIGHT

def getRandomColor():
    return (randint(COLORBOUNDS[0], COLORBOUNDS[1]), randint(COLORBOUNDS[0], COLORBOUNDS[1]), randint(COLORBOUNDS[0], COLORBOUNDS[1]))

def draw(image, drawing, offset = (0, 0), scale = (1, 1)):
    for stroke in drawing:
        points = np.array([np.array([int(xy[0] * scale[0]), int(xy[1] * scale[1])]) + offset for xy in zip(stroke[0], stroke[1]) if checkBounds(xy)]).reshape(-1, 1, 2)
        cv2.polylines(image, [points], isClosed=True, color = getRandomColor(), thickness=2)
    return image
        
def getRandomTransform():
    scaleX = (uniform(SCALELIMITS[0], SCALELIMITS[1]) * IMG_WIDTH) / (GRID_SIZE[0] * DRAWING_WIDTH)
    scaleY = (uniform(SCALELIMITS[0], SCALELIMITS[1]) * IMG_HEIGHT) /  (GRID_SIZE[1] * DRAWING_HEIGHT)
    return (scaleX, scaleY)

def randomSabotage():
    return uniform(0, 1) <= P_SABOTAGE

def process(drawing, image, data, boxnum):
    scale = getRandomTransform()
    rownum, colnum = boxnum // GRID_SIZE[1], (boxnum % GRID_SIZE[1])
    # print(scale, IMG_WIDTH, IMG_HEIGHT)
    fixedOffset = (colnum * CELL[0], rownum * CELL[1])
    limits = (int(CELL[0] * OFFSETLIMIT[0]), int(CELL[1] * OFFSETLIMIT[1]))
    randomOffset = (randint(-limits[0], limits[0]), randint(-limits[1], limits[1]))
    offset = (fixedOffset[0] + randomOffset[0], fixedOffset[1] + randomOffset[1])
    def bound(x0, y0, x1, y1):
        def boundValue(x, wall):
            x = min(x, wall)
            x = max(x, 0)
            return x
        return (boundValue(x0, IMG_WIDTH), boundValue(y0, IMG_HEIGHT), boundValue(x1, IMG_WIDTH), boundValue(y1, IMG_HEIGHT))

    data["boxes"].append(bound(offset[0], offset[1], int(offset[0] + DRAWING_WIDTH * scale[0]), int(offset[1] + DRAWING_HEIGHT * scale[1])))
    data["labels"].append(drawing["word"])
    draw(image, drawing['drawing'], offset, scale)
    return (image, data)
    
def createImage():
    image = np.full((IMG_HEIGHT, IMG_WIDTH, 3), 255, dtype = np.uint8)
    data = {"boxes" : [], "labels" : []}
    for i in range(GRID_SIZE[0] * GRID_SIZE[1]):
        if randomSabotage():
            continue
        drawing = choice(drawings)
        image, data = process(drawing, image, data, i)
    return image, data
    
def visualizeBBoxes(image, metadata, showBoxes = True):
    img = image.copy()
    print(metadata)
    if showBoxes:
        for obj in metadata:
            bbox = obj["bbox"]
            cv2.rectangle(img, (bbox[0], bbox[1]), (bbox[2], bbox[3]), (0, 0, 0), 2)
    cv2.imshow("BBox img", img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [7]:
img, data = createImage()

In [8]:
visualizeBBoxes(img, data, False)

{'boxes': [(33, 2, 189, 289), (136, 35, 290, 228), (287, 0, 512, 105), (349, 0, 504, 245), (7, 119, 220, 400), (278, 131, 458, 387), (415, 129, 512, 413), (0, 225, 175, 457), (274, 284, 512, 512), (388, 288, 512, 429), (131, 408, 307, 512), (262, 382, 433, 512), (371, 409, 512, 512)], 'labels': ['triangle', 'triangle', 'hexagon', 'octagon', 'circle', 'star', 'hexagon', 'square', 'line', 'square', 'star', 'line', 'triangle']}


In [9]:
from datasets import load_dataset, Dataset
from huggingface_hub import login

In [10]:
DATASET_SIZE = 20000
metadata = []
for i in tqdm(range(DATASET_SIZE)):
    img, data = createImage()
    filename = str(i)
    while len(filename) < 3:
        filename = "0" + filename
    filename += ".jpg"
    cv2.imwrite(f"./images/{filename}", img)
    imgdata = {}
    imgdata["file_name"] = filename
    imgdata["shapes"] = data
    metadata.append(imgdata)
    
with open("./images/metadata.jsonl", "w") as f:
    for item in metadata:
        f.write(json.dumps(item) + '\n')

100%|██████████| 20000/20000 [02:59<00:00, 111.73it/s]


In [11]:
dataset = load_dataset("imagefolder", data_dir='./images')

Resolving data files:   0%|          | 0/20001 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [12]:
login(token = "")

Token is valid (permission: write).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' credential helper as default.

git config --global credential.helper store

Read https://git-scm.com/book/en/v2/Git-Tools-Credential-Storage for more details.
Token has not been saved to git credential helper.
Your token has been saved to C:\Users\subha\.cache\huggingface\token
Login successful


In [14]:
dataset.push_to_hub("whENbhAI/doodle_512")

Uploading the dataset shards:   0%|          | 0/3 [00:00<?, ?it/s]

Map:   0%|          | 0/6667 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/67 [00:00<?, ?ba/s]

Map:   0%|          | 0/6667 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/67 [00:00<?, ?ba/s]

Map:   0%|          | 0/6666 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/67 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/whENbhAI/doodle_512/commit/fe931d7df213196f6d36018976271ebe46fa2fe5', commit_message='Upload dataset', commit_description='', oid='fe931d7df213196f6d36018976271ebe46fa2fe5', pr_url=None, pr_revision=None, pr_num=None)